In [6]:
# Import dependencies from Pandas, SKLearn, TensorFlow & MatPlotLib
from path import Path
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

import sklearn as skl
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,MinMaxScaler, LabelEncoder
from sklearn.svm import SVC

In [1]:
from sqlalchemy import create_engine

In [2]:
# Defining db info in config var
jdbc_url="jdbc:postgresql://34.67.52.115/team5k"
config = {'user': 'postgres', 
          "password": "team5kteam5k", 
          "driver":"org.postgresql.Driver",
          "location": "34.67.52.115",
          "db": "team5k",
          "port": "5432"}


#postgres://[user]:[password]@[location]:[port]/[database]
create_engine_str = 'postgresql://' + config["user"] + ":" + config["password"] + "@" + config["location"] + ":" + config["port"] + "/" + config["db"]
db = create_engine(create_engine_str)

In [9]:
#Test query
df=pd.read_sql("SELECT * FROM public.fec_donor_az LIMIT 5;",create_engine_str)
df.head()

,id,cmt_id,amndt_ind,rpt_tp,transaction_pgi,image_num,transaction_tp,entity_tp,last_name,first_name,...,employer,occupation,transaction_dt,transaction_amt,other_id,tran_id,file_num,memo_cd,memo_text,sub id
0,141712,c00310490,n,my,None,99020070054,15,None,might,thomas o,...,cableone,None,1999-06-16,1000,None,None,None,None,None,3061920110007090000
1,141713,c00342436,n,my,None,99020081109,15,None,roberts,susan,...,None,None,1999-03-10,1000,None,None,None,None,None,3061920110007110000
2,141714,c00287953,n,my,None,99020080810,15,None,baker,barry,...,sss corporation,None,1999-05-29,1000,None,None,None,None,None,3061920110007140000
3,141715,c00287953,n,my,None,99020080810,15,None,baker,alice,...,arizona mail order company inc,None,1999-05-29,1000,None,None,None,None,None,3061920110007140000
4,141716,c00287953,n,my,None,99020080810,15,None,baker,paul,...,arizona mail order company inc,None,1999-05-29,1000,None,None,None,None,None,3061920110007140000


In [11]:
#Union all donation tables
contrib_df=pd.read_sql_query('''SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, transaction_tp, entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, file_num, memo_cd, memo_text, "sub id"
	FROM public.fec_donor_az
union
SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, cast(transaction_tp as text), entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, file_num, memo_cd, memo_text, "sub id"
	FROM public.fec_donor_fl
union
SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, transaction_tp, entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, file_num, memo_cd, memo_text, "sub id"
	FROM public.fec_donor_mi
union
SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, cast(transaction_tp as text), entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, file_num, memo_cd, memo_text, "sub id"
	FROM public.fec_donor_nc
union
SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, transaction_tp, entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, file_num, memo_cd, memo_text, "sub id"
	FROM public.fec_donor_pa
union
SELECT id, cmt_id, amndt_ind, rpt_tp, transaction_pgi, image_num, transaction_tp, entity_tp, last_name, first_name, city, state, zip, employer, occupation, transaction_dt, transaction_amt, other_id, tran_id, cast(file_num as text), memo_cd, memo_text, "sub id"
	FROM public.fec_donor_wi''', create_engine_str)
contrib_df.head()

,id,cmt_id,amndt_ind,rpt_tp,transaction_pgi,image_num,transaction_tp,entity_tp,last_name,first_name,...,employer,occupation,transaction_dt,transaction_amt,other_id,tran_id,file_num,memo_cd,memo_text,sub id
0,374005,c00305342,n,ye,None,20035140709,15,None,stoltzfus,myron,...,None,None,1999-09-17,250,None,None,None,None,None,3061920110006920000
1,513944,c00305342,n,ye,None,20035140739,15,None,galbraith,john w,...,retired,None,1999-12-09,750,None,None,None,None,None,3061920110006980000
2,374077,c00343376,n,ye,None,20035230998,15,None,adezio,ron,...,gmr telecommunication,None,1999-12-17,500,None,None,None,None,None,3061920110006930000
3,240317,c00335729,n,ye,None,20035081187,15,None,zichichi,rose marie,...,ti-coating,None,1999-12-30,500,None,None,None,None,None,3061920110006610000
4,513965,c00331256,a,my,None,99034673585,15,None,bockorny,jill,...,housewife,None,1999-06-14,1000,None,None,None,None,None,3061920110007030000


In [14]:
#See Data Types
contrib_df.dtypes

id                   int64
cmt_id              object
amndt_ind           object
rpt_tp              object
transaction_pgi     object
image_num            int64
transaction_tp      object
entity_tp           object
last_name           object
first_name          object
city                object
state               object
zip                float64
employer            object
occupation          object
transaction_dt      object
transaction_amt      int64
other_id            object
tran_id             object
file_num            object
memo_cd             object
memo_text           object
sub id               int64
dtype: object

In [30]:
contrib_df.count()

id                 600
cmt_id             600
amndt_ind          600
rpt_tp             600
transaction_pgi     13
image_num          600
transaction_tp     600
entity_tp           89
last_name          600
first_name         600
city               599
state              600
zip                597
employer           540
occupation          91
transaction_dt     600
transaction_amt    600
other_id             1
tran_id             89
file_num            89
memo_cd              0
memo_text            2
sub id             600
dtype: int64

In [20]:
#Get Postal Code Info
zip_df=pd.read_sql("SELECT * FROM public.postal_codes;",create_engine_str)
zip_df.set_index('zip', inplace=True)
zip_df.head()

,county,state,stcountyfp,classfp
zip,,,,
36003,Autauga County,AL,1001,H1
36006,Autauga County,AL,1001,H1
36067,Autauga County,AL,1001,H1
36066,Autauga County,AL,1001,H1
36703,Autauga County,AL,1001,H1


In [21]:
zip_df.dtypes

county        object
state         object
stcountyfp     int64
classfp       object
dtype: object

In [36]:
contrib_county_df=contrib_df.join(zip_df, on='zip', how="left", lsuffix="_contrib", rsuffix="_postal")
contrib_county_df.head(50)
contrib_county_df.to_csv("test.csv")

In [33]:
contrib_county_df.dtypes

county              object
state_contrib       object
stcountyfp         float64
classfp             object
id                   int64
cmt_id              object
amndt_ind           object
rpt_tp              object
transaction_pgi     object
image_num            int64
transaction_tp      object
entity_tp           object
last_name           object
first_name          object
city                object
state_postal        object
zip                float64
employer            object
occupation          object
transaction_dt      object
transaction_amt      int64
other_id            object
tran_id             object
file_num            object
memo_cd             object
memo_text           object
sub id               int64
dtype: object

In [31]:
#Get County Votes
votes_df=pd.read_sql("SELECT * FROM public.pres_votes_6t;",create_engine_str)
votes_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Arizona,AZ,Apache,4001,President,Al Gore,democrat,13025,19456,20191203
1,2000,Arizona,AZ,Apache,4001,President,George W. Bush,republican,5947,19456,20191203
2,2000,Arizona,AZ,Apache,4001,President,Ralph Nader,green,245,19456,20191203
3,2000,Arizona,AZ,Apache,4001,President,Other,NA,239,19456,20191203
4,2000,Arizona,AZ,Cochise,4003,President,Al Gore,democrat,13360,33241,20191203


In [ ]:
#Get Commmitte Data
